In [4]:
import os 
from PIL import Image 
from arguments import parser 
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.manifold import TSNE
from datasets import create_dataset 
from torch.utils.data import DataLoader
from utils.metrics import MetricCalculator, loco_auroc
from accelerate import Accelerator
from omegaconf import OmegaConf
import seaborn as sns 
from main import torch_seed


torch_seed(42)
os.environ['CUDA_VISIBLE_DEVICES'] = '0' 
default_setting = './configs/default/mvtecad.yaml'
model_setting = './configs/model/cfgcad.yaml'
cfg = parser(True,default_setting, model_setting)


model  = __import__('models').__dict__[cfg.MODEL.method](
        backbone = cfg.MODEL.backbone,
        **cfg.MODEL.params
        ).to('cuda')
device = 'cuda'


loader_dict = {}
accelerator = Accelerator()
for cn in cfg.DATASET.class_names:
    trainset, testset = create_dataset(
        dataset_name  = cfg.DATASET.dataset_name,
        datadir       = cfg.DATASET.datadir,
        class_name    = cn,
        img_size      = cfg.DATASET.img_size,
        mean          = cfg.DATASET.mean,
        std           = cfg.DATASET.std,
        aug_info      = cfg.DATASET.aug_info,
        **cfg.DATASET.get('params',{})
    )
    trainloader = DataLoader(
        dataset     = trainset,
        batch_size  = cfg.DATASET.batch_size,
        num_workers = cfg.DATASET.num_workers,
        shuffle     = True 
    )    

    testloader = DataLoader(
            dataset     = testset,
            batch_size  = 8,
            num_workers = cfg.DATASET.num_workers,
            shuffle     = False 
        )    
    
    loader_dict[cn] = {'train':trainloader,'test':testloader}    


 Experiment Name : .-Continual_True-online_False



In [5]:
model = model.cuda()
back = model.backbone 
neck = model.neck 
rec = model.reconstruction 

In [6]:
with torch.no_grad():
    for images, labels, clslabel in trainloader: 
        Input = {'image':images.to('cuda'),'clslabel':clslabel.to('cuda')}
        
        Output = back(Input)
        Input.update(Output)
        
        Output = neck(Input)
        Input.update(Output)
        
        # Output = rec(Input,clslabel.to('cuda'))
        # Input.update(Output)
        
        break 

In [8]:
feature_align = Input['feature_align']
self = rec 
src, pos_embed = self.forward_pre(feature_align)

In [12]:
task_embedding.shape

torch.Size([16, 256])

torch.Size([32, 256])